In [199]:
# import libraries
import datetime
import googlemaps
import itertools
import json
import numpy as np
import pandas as pd
import pymysql
import re
import requests
import string
import swifter
import time
import urllib
from bs4 import BeautifulSoup as bs
from config import gkey
from IPython.display import clear_output
from requests import get
from scipy import stats
from selenium import webdriver
from splinter import Browser
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
from statsmodels.stats.proportion import proportions_ztest
from zodiac_sign import get_zodiac_sign

In [200]:
# Convert Player Data File
player_file = "Resources/player_data.csv"
nba_data = pd.read_csv(player_file)
nba_data.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [201]:
# Select specific columns from NBA player data to create new dataframe
nba_data = nba_data[['name', 'birth_date', 'height']].copy()

# Change the name of the columsn to match database names
nba_data.columns = ['player_name', 'birth_date', 'height']

# Add index as ID
nba_data['id'] = nba_data.index

In [202]:
# Add zodiac sign column to dataframe
nba_data['birth_date'] = pd.to_datetime(nba_data['birth_date'])
nba_data = nba_data.dropna()
nba_data['zodiac'] = nba_data['birth_date'].apply(get_zodiac_sign)
nba_data.head()

,player_name,birth_date,height,id,zodiac
0,Alaa Abdelnaby,1968-06-24,6-10,0,Cancer
1,Zaid Abdul-Aziz,1946-04-07,6-9,1,Aries
2,Kareem Abdul-Jabbar,1947-04-16,7-2,2,Aries
3,Mahmoud Abdul-Rauf,1969-03-09,6-1,3,Pisces
4,Tariq Abdul-Wahad,1974-11-03,6-6,4,Scorpio


In [203]:
# Create a unique ID for each player
nba_data['unique_id'] = nba_data['player_name']+'_'+(nba_data['birth_date'].astype('str'))
nba_data.head()

,player_name,birth_date,height,id,zodiac,unique_id
0,Alaa Abdelnaby,1968-06-24,6-10,0,Cancer,Alaa Abdelnaby_1968-06-24
1,Zaid Abdul-Aziz,1946-04-07,6-9,1,Aries,Zaid Abdul-Aziz_1946-04-07
2,Kareem Abdul-Jabbar,1947-04-16,7-2,2,Aries,Kareem Abdul-Jabbar_1947-04-16
3,Mahmoud Abdul-Rauf,1969-03-09,6-1,3,Pisces,Mahmoud Abdul-Rauf_1969-03-09
4,Tariq Abdul-Wahad,1974-11-03,6-6,4,Scorpio,Tariq Abdul-Wahad_1974-11-03


In [204]:
df = pd.read_csv("NCCA_Players_Final-1.csv")
df.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,college,dob,height,high_school,link,name,pob,pos,season,...,FTM,GP,GS,MIN,OFF,PF,PTS,STL,TOV,TRB
0,0,Arizona State,19880819,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2009,...,1.15,130,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88
1,3,Arizona State,19880819,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2010,...,1.15,130,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88
2,6,Arizona State,19880819,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2011,...,1.15,130,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88
3,9,UCLA,19880602,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2009,...,0.08,51,1.0,6.3,0.08,0.45,1.49,0.12,0.43,0.41
4,11,UCLA,19880602,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2010,...,0.08,51,1.0,6.3,0.08,0.45,1.49,0.12,0.43,0.41


In [206]:
# Drop unnamed index column
df = df.drop(['Unnamed: 0'], axis=1)

In [207]:
df['dob'] = pd.to_datetime(df['dob'], format='%Y%m%d')
df.head()

,college,dob,height,high_school,link,name,pob,pos,season,tenure,...,FTM,GP,GS,MIN,OFF,PF,PTS,STL,TOV,TRB
0,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2009,4.0,...,1.15,130,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88
1,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2010,6.0,...,1.15,130,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88
2,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2011,8.0,...,1.15,130,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88
3,UCLA,1988-06-02,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2009,5.0,...,0.08,51,1.0,6.3,0.08,0.45,1.49,0.12,0.43,0.41
4,UCLA,1988-06-02,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2010,6.0,...,0.08,51,1.0,6.3,0.08,0.45,1.49,0.12,0.43,0.41


In [208]:
df['zodiac'] = df['dob'].apply(get_zodiac_sign)

In [209]:
# Create a unique ID for each player
df['unique_id'] = df['name']+'_'+(df['dob'].astype('str'))
df = df.drop_duplicates()
df.head()

,college,dob,height,high_school,link,name,pob,pos,season,tenure,...,GS,MIN,OFF,PF,PTS,STL,TOV,TRB,zodiac,unique_id
0,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2009,4.0,...,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88,Leo,Ty Abbott_1988-08-19
1,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2010,6.0,...,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88,Leo,Ty Abbott_1988-08-19
2,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2011,8.0,...,120.0,30.9,0.62,2.42,10.23,1.02,1.53,3.88,Leo,Ty Abbott_1988-08-19
3,UCLA,1988-06-02,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2009,5.0,...,1.0,6.3,0.08,0.45,1.49,0.12,0.43,0.41,Gemini,Mustafa Abdul-Hamid_1988-06-02
4,UCLA,1988-06-02,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2010,6.0,...,1.0,6.3,0.08,0.45,1.49,0.12,0.43,0.41,Gemini,Mustafa Abdul-Hamid_1988-06-02


In [213]:
nba_player_ids = nba_data['unique_id']

In [214]:
overlap = df.loc[df['unique_id'].isin(nba_player_ids)]
len(overlap)

1390

In [215]:
undrafted = df.loc[~df['unique_id'].isin(nba_player_ids)]
len(undrafted)

50289

In [216]:
undrafted = undrafted.sort_values('season', ascending=False).drop_duplicates('unique_id',keep='first').sort_index()

In [217]:
undrafted['drafted'] = 0

In [218]:
undrafted.head()

,college,dob,height,high_school,link,name,pob,pos,season,tenure,...,MIN,OFF,PF,PTS,STL,TOV,TRB,zodiac,unique_id,drafted
2,Arizona State,1988-08-19,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2011,8.0,...,30.9,0.62,2.42,10.23,1.02,1.53,3.88,Leo,Ty Abbott_1988-08-19,0
4,UCLA,1988-06-02,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2010,6.0,...,6.3,0.08,0.45,1.49,0.12,0.43,0.41,Gemini,Mustafa Abdul-Hamid_1988-06-02,0
5,Mercer,1940-01-01,69,NaN,/player/Larney-Abdush-Shakur/Summary/14491,Larney Abdush-Shakur,Austell (GA),PG,2009,4.0,...,2.1,0.07,0.21,0.36,0.21,0.14,0.29,Capricorn,Larney Abdush-Shakur_1940-01-01,0
8,Sacred Heart,1940-01-01,77,Marianapolis Preparatory School,/player/Toso-Abedamowo/Summary/20808,Toso Abedamowo,Lincoln (RI),G,2011,6.0,...,2.3,0.00,0.17,0.67,0.33,0.17,0.17,Capricorn,Toso Abedamowo_1940-01-01,0
11,Texas Southern,1987-03-04,79,Creative Mind Foundation,/player/Onyekautukwu-Abiakam/Summary/15292,Onyekautukwu Abiakam,Lagos,F,2011,7.0,...,14.1,1.05,1.83,2.93,0.31,0.66,3.22,Pisces,Onyekautukwu Abiakam_1987-03-04,0


In [219]:
overlap.head()

,college,dob,height,high_school,link,name,pob,pos,season,tenure,...,GS,MIN,OFF,PF,PTS,STL,TOV,TRB,zodiac,unique_id
37,Baylor,1990-10-06,79,John Horn High School,/player/Quincy-Acy/Summary/4130,Quincy Acy,Tyler (TX),F,2009,2.0,...,16.0,1.73,2.58,0.65,NaN,0.40,4.87,0.64,Libra,Quincy Acy_1990-10-06
38,Baylor,1990-10-06,79,John Horn High School,/player/Quincy-Acy/Summary/4130,Quincy Acy,Tyler (TX),F,2010,4.0,...,16.0,1.73,2.58,0.65,NaN,0.40,4.87,0.64,Libra,Quincy Acy_1990-10-06
39,Baylor,1990-10-06,79,John Horn High School,/player/Quincy-Acy/Summary/4130,Quincy Acy,Tyler (TX),F,2011,6.0,...,16.0,1.73,2.58,0.65,NaN,0.40,4.87,0.64,Libra,Quincy Acy_1990-10-06
40,Baylor,1990-10-06,79,John Horn High School,/player/Quincy-Acy/Summary/4130,Quincy Acy,Tyler (TX),F,2012,8.0,...,16.0,1.73,2.58,0.65,NaN,0.40,4.87,0.64,Libra,Quincy Acy_1990-10-06
75,Connecticut,1986-02-10,79,Brewster Academy,/player/Jeff-Adrien/Summary/1641,Jeff Adrien,Brookline (MA),SF,2009,8.0,...,14.0,1.73,2.86,0.59,NaN,0.52,4.63,0.65,Aquarius,Jeff Adrien_1986-02-10


In [220]:
overlap = overlap.drop(["3P%","3PA","3PM","AST","BLK","DEF","FG%","FGA",
               "FGM","FTA","FTM","GP","GS","MIN","OFF","PF","PTS","STL","TOV","TRB", 'FT%'], axis=1)

In [221]:
ncaa_nba_converts = pd.read_csv("NBA_Converts_Stats.csv")

In [222]:
ncaa_nba_converts

,3P%,3PA,3PM,AST,BLK,DEF,FG%,FGA,FGM,FT%,...,GS,MIN,OFF,PF,PTS,STL,TOV,TRB,name,unique_id
0,0.324,2.7,0.9,0.9,0.7,3.8,0.455,10.2,4.7,0.761,...,56.0,27.9,2.5,2.7,13.8,0.6,1.6,6.3,Ben Bentil,Ben Bentil_1995-03-29
1,0.265,1.4,0.4,5.2,0.4,4.8,0.425,4.3,1.8,0.617,...,129.0,30.9,0.7,2.3,5.8,1.5,1.9,5.5,Julyan Stone,Julyan Stone_1988-12-07
2,0.412,5.4,2.2,7.6,0.8,5.1,0.551,9.5,5.3,0.673,...,36.0,35.1,0.9,1.8,14.6,1.8,2.5,6.0,Lonzo Ball,Lonzo Ball_1997-10-27
3,0.359,4.7,1.7,1.7,0.2,2.3,0.377,6.2,2.3,0.733,...,90.0,26.5,0.7,1.1,7.2,0.8,0.8,3.0,Chris Babb,Chris Babb_1990-02-14
4,0.071,0.2,0.0,1.2,3.4,6.1,0.518,10.8,5.6,0.669,...,65.0,33.8,2.7,2.4,14.0,0.8,2.8,8.8,Jordan Mickey,Jordan Mickey_1994-07-09
5,NaN,0.0,0.0,0.8,1.4,5.1,0.578,8.8,5.1,0.636,...,79.0,27.0,2.8,2.7,12.5,0.5,2.0,7.9,Jordan Hill,Jordan Hill_1987-07-27
6,0.372,4.0,1.5,1.9,0.2,3.3,0.443,12.5,5.5,0.734,...,109.0,29.0,0.8,2.1,15.8,0.8,3.0,4.2,Kevin Murphy,Kevin Murphy_1990-03-06
7,0.000,0.0,0.0,1.0,2.1,5.4,0.611,8.1,5.0,0.629,...,35.0,27.5,3.0,2.6,12.4,0.9,2.5,8.4,Derrick Favors,Derrick Favors_1991-07-15
8,0.331,4.3,1.4,2.1,0.2,3.3,0.467,11.6,5.4,0.839,...,63.0,30.2,1.3,2.1,16.4,2.4,1.4,4.6,Jordan Adams,Jordan Adams_1994-07-08
9,0.387,4.2,1.6,1.5,0.2,4.0,0.471,11.6,5.4,0.813,...,33.0,29.8,1.3,2.2,15.6,0.9,1.7,5.3,Malik Beasley,Malik Beasley_1996-11-26


In [223]:
overlap = overlap.sort_values('season', ascending=False).drop_duplicates('unique_id',keep='first').sort_index()

In [224]:
# Merge two dataframes to get open date for all jobs
nba_converts_full = pd.merge(ncaa_nba_converts, overlap[['pob','college','height',
                                                         'high_school','pos','season','tenure',
                                                         'weight','unique_id', 'dob', 'link', 'name_dob', 'zodiac']], on='unique_id', how="left")

In [225]:
nba_converts_full.head()

,3P%,3PA,3PM,AST,BLK,DEF,FG%,FGA,FGM,FT%,...,height,high_school,pos,season,tenure,weight,dob,link,name_dob,zodiac
0,0.324,2.7,0.9,0.9,0.7,3.8,0.455,10.2,4.7,0.761,...,81,St. Andrew's School,F,2016.0,4.0,235,1995-03-29,/player/Ben-Bentil/Summary/80300,Ben Bentil_19950329,Aries
1,0.265,1.4,0.4,5.2,0.4,4.8,0.425,4.3,1.8,0.617,...,78,Dos Pueblos High School,PG,2011.0,8.0,200,1988-12-07,/player/Julyan-Stone/Summary/8270,Julyan Stone_19881207,Sagittarius
2,0.412,5.4,2.2,7.6,0.8,5.1,0.551,9.5,5.3,0.673,...,78,Chino Hills High School,PG,2017.0,2.0,190,1997-10-27,/player/Lonzo-Ball/Summary/59501,Lonzo Ball_19971027,Scorpio
3,0.359,4.7,1.7,1.7,0.2,2.3,0.377,6.2,2.3,0.733,...,77,The Oakridge School,G,2013.0,8.0,225,1990-02-14,/player/Chris-Babb/Summary/3969,Chris Babb_19900214,Aquarius
4,0.071,0.2,0.0,1.2,3.4,6.1,0.518,10.8,5.6,0.669,...,80,Grace Preparatory Academy,PF,2015.0,4.0,235,1994-07-09,/player/Jordan-Mickey/Summary/28236,Jordan Mickey_19940709,Cancer


In [226]:
nba_converts_full['drafted'] = 1

In [227]:
nba_converts_full.head()

,3P%,3PA,3PM,AST,BLK,DEF,FG%,FGA,FGM,FT%,...,high_school,pos,season,tenure,weight,dob,link,name_dob,zodiac,drafted
0,0.324,2.7,0.9,0.9,0.7,3.8,0.455,10.2,4.7,0.761,...,St. Andrew's School,F,2016.0,4.0,235,1995-03-29,/player/Ben-Bentil/Summary/80300,Ben Bentil_19950329,Aries,1
1,0.265,1.4,0.4,5.2,0.4,4.8,0.425,4.3,1.8,0.617,...,Dos Pueblos High School,PG,2011.0,8.0,200,1988-12-07,/player/Julyan-Stone/Summary/8270,Julyan Stone_19881207,Sagittarius,1
2,0.412,5.4,2.2,7.6,0.8,5.1,0.551,9.5,5.3,0.673,...,Chino Hills High School,PG,2017.0,2.0,190,1997-10-27,/player/Lonzo-Ball/Summary/59501,Lonzo Ball_19971027,Scorpio,1
3,0.359,4.7,1.7,1.7,0.2,2.3,0.377,6.2,2.3,0.733,...,The Oakridge School,G,2013.0,8.0,225,1990-02-14,/player/Chris-Babb/Summary/3969,Chris Babb_19900214,Aquarius,1
4,0.071,0.2,0.0,1.2,3.4,6.1,0.518,10.8,5.6,0.669,...,Grace Preparatory Academy,PF,2015.0,4.0,235,1994-07-09,/player/Jordan-Mickey/Summary/28236,Jordan Mickey_19940709,Cancer,1


In [230]:
final_set = pd.concat([undrafted, nba_converts_full])

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if __name__ == '__main__':


In [231]:
final_set.head()

,3P%,3PA,3PM,AST,BLK,DEF,FG%,FGA,FGM,FT%,...,link,name,name_dob,pob,pos,season,tenure,unique_id,weight,zodiac
2,0.360,5.83,2.10,1.80,0.43,3.27,0.398,8.77,3.49,0.700,...,/player/Ty-Abbott/Summary/6680,Ty Abbott,Ty Abbott_19880819,Phoenix (AZ),G,2011.0,8.0,Ty Abbott_1988-08-19,209,Leo
4,0.324,0.73,0.24,0.55,0.00,0.33,0.448,1.31,0.59,0.364,...,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,Mustafa Abdul-Hamid_19880602,St. Louis (MO),G,2010.0,6.0,Mustafa Abdul-Hamid_1988-06-02,195,Gemini
5,0.333,0.21,0.07,0.07,0.00,0.21,0.182,0.79,0.14,0.000,...,/player/Larney-Abdush-Shakur/Summary/14491,Larney Abdush-Shakur,Larney Abdush-Shakur_19400101,Austell (GA),PG,2009.0,4.0,Larney Abdush-Shakur_1940-01-01,153,Capricorn
8,0.333,0.50,0.17,0.00,0.00,0.17,0.333,0.50,0.17,0.500,...,/player/Toso-Abedamowo/Summary/20808,Toso Abedamowo,Toso Abedamowo_19400101,Lincoln (RI),G,2011.0,6.0,Toso Abedamowo_1940-01-01,190,Capricorn
11,0.000,0.00,0.00,0.28,1.10,2.17,0.497,2.57,1.28,0.468,...,/player/Onyekautukwu-Abiakam/Summary/15292,Onyekautukwu Abiakam,Onyekautukwu Abiakam_19870304,Lagos,F,2011.0,7.0,Onyekautukwu Abiakam_1987-03-04,195,Pisces


In [55]:
final_set.to_csv("NCAA_Conversions_Final.csv")

In [232]:
copy = final_set

In [155]:
# Get list of birth cities from dataframe
cities = copy['pob'].unique()

# Use googlemaps API to get lat and lng for birth city
gmaps = googlemaps.Client(key=gkey)

# for each row's birth city get the latitude and longitude and add it to the dataframe
coordinates = {}
for location in cities:
    city = gmaps.geocode(location)
    try:
        lat = city[0]['geometry']['location']['lat']
    except:
        pass
    try:
        long = city[0]['geometry']['location']['lng']
    except:
        pass
    coordinates[location] = {'lat': lat, 'long':long}

In [101]:
city_data = pd.DataFrame.from_dict(coordinates, orient='index')

In [102]:
city_data = city_data.reset_index()

In [156]:
city_data.columns = ['pob','lat', 'long']

In [233]:
final_copy = pd.merge(copy, city_data, how='left', on="pob")

In [234]:
final_copy.head()

,3P%,3PA,3PM,AST,BLK,DEF,FG%,FGA,FGM,FT%,...,name_dob,pob,pos,season,tenure,unique_id,weight,zodiac,lat,long
0,0.360,5.83,2.10,1.80,0.43,3.27,0.398,8.77,3.49,0.700,...,Ty Abbott_19880819,Phoenix (AZ),G,2011.0,8.0,Ty Abbott_1988-08-19,209,Leo,33.448377,-112.074037
1,0.324,0.73,0.24,0.55,0.00,0.33,0.448,1.31,0.59,0.364,...,Mustafa Abdul-Hamid_19880602,St. Louis (MO),G,2010.0,6.0,Mustafa Abdul-Hamid_1988-06-02,195,Gemini,38.627003,-90.199404
2,0.333,0.21,0.07,0.07,0.00,0.21,0.182,0.79,0.14,0.000,...,Larney Abdush-Shakur_19400101,Austell (GA),PG,2009.0,4.0,Larney Abdush-Shakur_1940-01-01,153,Capricorn,33.812606,-84.634378
3,0.333,0.50,0.17,0.00,0.00,0.17,0.333,0.50,0.17,0.500,...,Toso Abedamowo_19400101,Lincoln (RI),G,2011.0,6.0,Toso Abedamowo_1940-01-01,190,Capricorn,41.911012,-71.441810
4,0.000,0.00,0.00,0.28,1.10,2.17,0.497,2.57,1.28,0.468,...,Onyekautukwu Abiakam_19870304,Lagos,F,2011.0,7.0,Onyekautukwu Abiakam_1987-03-04,195,Pisces,6.524379,3.379206


In [235]:
final_copy.to_csv("NCAA_Conversions_031319.csv")